<a href="https://colab.research.google.com/github/carlocamilloni/Structural-Bioinformatics/blob/main/Notebooks/t07_StructurePred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protein Structure Prediction

In this notebook you will try to characterize one protein from its sequence using multiple tools. In particular we will work on proteins from Cyprinid herpesvirus 2 that is a virus attacking gold fishes. You will perform

1.   Secondary structure prediction
2.   Tertiary structure prediction

As for the last time, link your google drive, create a folder for this  task and save a copy of this notebook there

Then move to your local folder:

In [ ]:
# mount google drive  
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
local_path='/content/drive/MyDrive/Structural_Bioinformatics/Task5'   # PASTE YOUR LOCAL PATH IN PLACE OF /content
%cd {local_path}

Now you can choose one among these sequences on which you will work. To choose it set the seq variable in the following:

*   A0A0E3XAL1: Epimerase domain-containing protein
*   A0A0E3XA07: Uncharacterized protein
*   A0A0E3X9M4: Deoxyuridine triphosphatase
*   A0A0E3T6V3: FXYD domain-containing ion transport regulator

You can learn more about these proteins checking them on www.uniprot.org


In [ ]:
# XXX is the protein you choose
seq='XXX.fasta'

In [ ]:
#get the sequence you choose as:
!wget https://www.uniprot.org/uniprot/{seq}

In [ ]:
#Installing biopython using pip
!pip install biopython

Biopython is an excellent companion for working with DNA and protein sequences and also with structures. Here, we will show how to use it for reading a protein sequence file.


In [ ]:
import os
from pathlib import Path 
from Bio import SeqIO, Entrez
# read your protein sequence
aaseq = SeqIO.read(seq, format="fasta")

What is great about SeqIO is that you can use it to manipulate your sequence (e.g. sorting, changing formats, etc) and also to print information about your sequence, such as its description, sequence and accession ID.
You can try these commands below by first writing "aaseq." and then selecting one of the autocomplete options suggested by Google Colab. With this information, try to obtain the information requested below.

In [ ]:
#Printing the number of amino acids as an example
print("Sequence length (aa):")
print(len(aaseq))

In [ ]:
#Printing the description of the sequence
print("The description of the sequence is:")
print(aaseq.description)

#Printing the accession ID of the sequence
print("The ID of the sequence is:")
print(aaseq.id)

#Printing the sequence
print("The sequence is:")
print(aaseq.seq)

## Secondary Structure Prediction

Now we can begin with a secondary structure prediction from the sequence. For this we will use S4PRED (https://github.com/psipred/s4pred)

In [ ]:
%cd {local_path}

In [ ]:
# we need to download and install s4pred
!git clone --depth 1 https://github.com/psipred/s4pred
%cd s4pred
!wget http://bioinfadmin.cs.ucl.ac.uk/downloads/s4pred/weights.tar.gz
!tar -xvzf weights.tar.gz

In [ ]:
# Install: PyTorch
# this can be long...
!pip install torch==1.11.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

Now we can run the prediction over our fasta sequence and save it in a text file. You can then double click on the file to see the results

In [ ]:
%cd {local_path}
!python s4pred/run_model.py {seq} > ss.{seq}.txt
%ls ss.{seq}.txt

The file is made of three columns that report the secondary structure prediction and are COIL/HELIX/EXTENDED, we can now load and plot the results

In [ ]:
import pandas as pd
# here we are giving the titles to the columns of the file
header_list = ["position", "sequence", "ss", "coil", "helix","extended"]
# here we are reading the file skipping the first two lines
# you need to set your file name
data = pd.read_csv('ss.XXX.fasta.txt',sep='\s+',skiprows=2,names=header_list)
data = pd.DataFrame(data)

import matplotlib.pyplot as plt
#we are generating three plots calling each column with its name
#and setting the properties of the plot (e.g. the color of the line)
plt.plot(data['position'], data['coil'], color='black')
plt.plot(data['position'], data['helix'], color='blue')
plt.plot(data['position'], data['extended'], color='red')
#show the plot now
plt.show()

you can check how to improve the plot looking for matplotlib plot example. This plot can be used to describe what the secondary structure of the protein can be

## Structure Prediction


Here you will try to predict your structure using alphafold. Prepare a folder to save your data. Open the google colab at this link 
https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb
and save it locally before moving on



Depending on the length of your sequence alphafold can take around 30 minutes. 

Once you have all the PDB structures you can compare them using VMD.

#Report

Start by creating a new notebook, using File -> New Notebook. New notebooks are created in your google drive, in a Colab Notebooks folder.

When you are in the new notebook, generally named "Untitled.ipynb", select File -> Move and move it to your Task5 folder.

Rename the notebook as Report_Task5.ipynb

You can then copy and paste the following cells and then modify them as appropriate. The resulting notebook will be your first report.

# Protein Structure Prediction

your name

In [ ]:

from datetime import date

today = date.today()
print(today)
     

## Primary Structure

Describe here what you know about your protein and show your sequence

...

## Secondary Structure

Add the plot about the secondary struture prediction and briefly describe it

## Tertiary Structure

Upload your predicted PDB structure in your TASK5 folder so that you can display interactively here:

In [ ]:
!pip install py3Dmol

In [ ]:
#@title Display 3D structure {run: "auto"}
import py3Dmol
import glob
import matplotlib.pyplot as plt
# number of models to use
rank_num = 2 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

pdb_filename = f"*_rank_{rank_num}_*.pdb"

pdb_file = glob.glob(pdb_filename)

def show_pdb(rank_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"rank_{rank_num}"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file[0],'r').read(),'pdb')


  view.setStyle({'cartoon': {'color':'spectrum'}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view


show_pdb(rank_num,show_sidechains, show_mainchains).show()

Describe the results of your prediction: do they agree with the secondary structure prediction? Can you evaluate the quality of the prediction looking at the plots resulting from ColabFOLD?

